In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

data = pd.read_csv('shop_detail.csv')


current_time = data['mtime'].max()
data['shop_age_days'] = (current_time - data['mtime']) / (86400) 


data['log_follower_count'] = np.log1p(data['follower_count'])
data['log_rating_good'] = np.log1p(data['rating_good']) 


features = ['item_count', 'rating_star', 'response_rate', 'shop_age_days', 'log_rating_good']
target = 'log_follower_count'


data_clean = data[features + [target]].dropna()

print("Dữ liệu sau khi xử lý:")
print(data_clean.head())

Dữ liệu sau khi xử lý:
   item_count  rating_star  response_rate  shop_age_days  log_rating_good  \
0          56     4.889908           48.0       0.109340         4.691348   
1         137     4.841612           93.0       0.062928         7.659643   
2         197     4.887417           87.0       0.062870         6.820016   
3          83     4.886266           98.0       0.135116         6.139885   
4         334     4.870466           91.0       1.101910         5.241747   

   log_follower_count  
0            7.268920  
1            9.459619  
2            7.474772  
3            7.711997  
4            7.784473  


In [ ]:

X = data_clean[features]
y = data_clean[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LinearRegression()
model.fit(X_train, y_train)


y_pred_log = model.predict(X_test)


y_pred_real = np.expm1(y_pred_log) 
y_test_real = np.expm1(y_test)


r2 = r2_score(y_test, y_pred_log) 
print(f"Độ chính xác (R-squared) trên tập Test: {r2:.4f}")


coef_data = pd.DataFrame({'Feature': features, 'Coefficient': model.coef_})
print("\nMức độ ảnh hưởng của từng yếu tố:")
print(coef_data)

Độ chính xác (R-squared) trên tập Test: 0.6540

Mức độ ảnh hưởng của từng yếu tố:
           Feature  Coefficient
0       item_count    -0.000062
1      rating_star    -1.102297
2    response_rate     0.012192
3    shop_age_days    -0.001972
4  log_rating_good     0.637971


In [2]:
from mpl_toolkits.mplot3d import Axes3D 


X_viz_feature1 = data_clean['log_rating_good']          
X_viz_feature2 = np.log1p(data_clean['item_count'])     
Z_target = data_clean['log_follower_count']             


model_viz = LinearRegression()

X_viz = pd.DataFrame({'f1': X_viz_feature1, 'f2': X_viz_feature2})
model_viz.fit(X_viz, Z_target)


x_range = np.linspace(X_viz_feature1.min(), X_viz_feature1.max(), 30)
y_range = np.linspace(X_viz_feature2.min(), X_viz_feature2.max(), 30)
x_surf, y_surf = np.meshgrid(x_range, y_range)


z_surf = model_viz.intercept_ + (model_viz.coef_[0] * x_surf) + (model_viz.coef_[1] * y_surf)


fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')


ax.scatter(X_viz_feature1, X_viz_feature2, Z_target, c='blue', marker='o', alpha=0.3, label='Shop Thực tế')


ax.plot_surface(x_surf, y_surf, z_surf, color='red', alpha=0.4)


ax.set_xlabel('Log Rating Good (Uy tín)')
ax.set_ylabel('Log Item Count (Quy mô)')
ax.set_zlabel('Log Follower Count (Thành công)')
ax.set_title('Biểu đồ 3D: Tác động của Rating & Sản phẩm đến Follower')
ax.view_init(elev=20, azim=-60) 

plt.legend()
plt.show()

NameError: name 'data_clean' is not defined